### 1.导入必要的包

In [77]:
import quantmind as qm
import pandas as pd
import datetime
import backtrader as bt
import math
import numpy as np
import random
random.seed(2019)
#from tabulate import tabulate

### 2.对股票数据预处理，剔除亏损过高的股票，并随机选取一定数量股票进行试验

In [78]:
g_init_cash = 1000000

#all_stk：股票清单表，有code，name，area，market，list_date五行
all_stk = qm.get_stock_list()
all_stk['list_date']=pd.to_datetime(list(all_stk['list_date']),format="%Y%m%d")
# 剔除上市时间短的股票
all_stk = all_stk[(all_stk['market']!='科创板') & (all_stk['list_date']<'20170101')]

# 剔除ST：连续三年亏损的股票
# codes是股票代码合集
codes = list(set(all_stk['code'])-set(all_stk[all_stk['name'].str.contains('ST')]['code']))
stock_number=100
codes=random.sample(codes,stock_number)
print('预处理后剩余股票的支数：',len(codes))

预处理后剩余股票的支数： 100


### 3.准备指数数据

In [79]:
def prepare_index_data(start_date,end_date):
    #导入指数的日数据，包括开盘价open，最高价high，最低价low，收盘价close，发行总量volume
    index_day = qm.get_bar('000001.SH', start_date=start_date,end_date=end_date, freq='1d')
    #调整日期格式，将20200101调整为2020-01-01
    index_day['trade_date'] = pd.to_datetime(index_day['trade_date'],format="%Y%m%d")
    #向cerebro中导入：datatime列-trade_date；open、high、low、close、colume列对应；open_interest未平仓合约列自动生成
    dataa = bt.feeds.PandasData(
                dataname=index_day,
                datetime = 'trade_date',
                open='open',
                high='high',
                low='low',
                close='close',
                volume='volume',
                openinterest=-1,
                timeframe=bt.TimeFrame.Days,
               )
    cerebro.adddata(dataa, name='index_day')

### 4.对所有用到的股票生成常用的均线等数据

In [80]:
class TradeAnalyzer(bt.Analyzer):
    """
    ref - bt's unique trade identifier
    ticker - data feed name
    datein - date and time of trade opening
    pricein - price of trade entry                      买入价
    dir - long or short
    dateout - date and time of trade closing
    priceout - price of trade exit                      卖出价
    chng% - exit price to entry price ratio             卖出价相对买入价的盈利百分比
    pnl - money profit/loss per trade                   盈亏
    pnl% - proft/loss in %s to broker's value at the trade closing  盈亏占交易关闭时账户总值的百分比
    cumpnl - cumulative profit/loss                     累计的盈亏
    size - max position size during trade               交易中最大的成交数量
    value - max trade value                             交易中最大的成交总值
    nbars - trade duration in bars                      交易持续的bar数
    pnl/bar - profit/loss per bar                       平均每根bar的盈亏
    mfe% - max favorable excursion                      最大有利变动幅度
    mae% - max adverse excursion                        最大不利变动幅度
    """

    def get_analysis(self):
        return self.trades

    def __init__(self):
        self.trades = []
        self.cumprofit = 0.0
        
    # 准备上述的所有数据
    def notify_trade(self, trade):
        if trade.isclosed:
            brokervalue = self.strategy.broker.getvalue()
            dir = 'sell'
            if trade.history[0].event.size > 0: dir = 'buy'

            pricein = trade.history[len(trade.history) - 1].status.price
            priceout = trade.history[len(trade.history) - 1].event.price
            datein = bt.num2date(trade.history[0].status.dt)
            dateout = bt.num2date(trade.history[len(trade.history) - 1].status.dt)
            if trade.data._timeframe >= bt.TimeFrame.Days:
                datein = datein.date()
                dateout = dateout.date()

            pcntchange = 100 * priceout / pricein - 100
            pnl = trade.history[len(trade.history) - 1].status.pnlcomm
            pnlpcnt = 100 * pnl / brokervalue
            barlen = trade.history[len(trade.history) - 1].status.barlen
            pbar = pnl / (barlen+1)
            self.cumprofit += pnl

            size = value = 0.0
            for record in trade.history:
                if abs(size) < abs(record.status.size):
                    size = record.status.size
                    value = record.status.value

            highest_in_trade = max(trade.data.high.get(ago=0, size=barlen + 1))
            lowest_in_trade = min(trade.data.low.get(ago=0, size=barlen + 1))
            hp = 100 * (highest_in_trade - pricein) / pricein
            lp = 100 * (lowest_in_trade - pricein) / pricein
            if dir == 'buy':
                mfe = hp
                mae = lp
            if dir == 'sell':
                mfe = -lp
                mae = -hp

            self.trades.append({'ref': trade.ref, 'ticker': trade.data._name,
                                '买入日期': datein, '买入价格': pricein, '卖出日期': dateout, '卖出价格': priceout,
                                'chng%': round(pcntchange, 2), 'pnl': pnl, 'pnl%': round(pnlpcnt, 2),
                                'size': size, 'value': value, 'cumpnl': self.cumprofit,
                                'nbars': barlen, 'pnl/bar': round(pbar, 2),
                                'mfe%': round(mfe, 2), 'mae%': round(mae, 2)})

In [81]:
def prepare_stk_data(codes,start_date,end_date):
    stock_data = dict()
    start_date2 = pd.to_datetime(start_date,format="%Y%m%d")-datetime.timedelta(days=350)
    start_date2 = start_date2.strftime('%Y%m%d')
    print('准备数据中……')
    for stk in codes:
#         try:
#             list_date = all_stk[all_stk['code']==stk].list_date.iloc[0]
#             days = (pd.to_datetime(start_date,format="%Y%m%d")-list_date).days
#         except:
#             days = 0
#         print(days)    
#         if days<500:
#             continue
        
        
        try:
            stk_datas = qm.get_bar(stk,start_date=start_date2 ,end_date=end_date,freq='1d')
            stk_datas['ma_5'] = list(stk_datas['close'].rolling(5).mean().shift(1))
            stk_datas['ma_10'] = list(stk_datas['close'].rolling(10).mean().shift(1))
            stk_datas['ma_20'] = list(stk_datas['close'].rolling(20).mean().shift(1))
            stk_datas['ma_60'] = list(stk_datas['close'].rolling(60).mean().shift(1))
            stk_datas['ma_120'] = list(stk_datas['close'].rolling(120).mean().shift(1))
            stk_datas['ma_year'] = list(stk_datas['close'].rolling(250).mean().shift(1))

            stk_datas['trade_date'] = pd.to_datetime(stk_datas['trade_date'],format="%Y%m%d")
            stk_datas = stk_datas[stk_datas['trade_date']>=pd.to_datetime(start_date,format="%Y%m%d")]
            stk_datas.index = stk_datas.trade_date
        except:
            stk_datas = None
#     在交易期间一直停牌的
        if (stk_datas is None) or (stk_datas.empty):
            continue
            
        stock_data[stk] = stk_datas
#     np.save('daily_datas.npy', stock_data)
#     stock_data = np.load('2021_all_datas.npy', allow_pickle=True).item()

        
    return stock_data

In [82]:
#函数用途：止盈止损卖股票，编写卖出逻辑
def sell_stk(self,stk):
    ##仓位
    stk_size=self.getposition(stk).size
    
    #持仓成本
#     if stk_size>0:
    if stk_size>0:#self.hold[stk] is not None and 
        
        cur_date = stk.datetime.datetime()
        
        cost = stk_size * self.getposition(stk).price
        # 当前市值
        current_value = stk_size * stk.close[0]
        # 持仓利润
        profit = current_value - cost
        profit_percent = (profit / cost) * 100.0
        
#         cur_date = stk.datetime.datetime()
        
#         if (cur_date-self.hold[stk]).days>0:
       
    
        #条件7:macd值显著小于上一个高点的macd值
        if (stk.close[0]>=self.hest[stk][-1]) and (self.macd[stk][0]<self.macd[stk][-1]):
            self.log('-动力不足，调出: [%s], 盈亏:%.2f元(%.2f%%), 数量：(%d股)' % (stk._name, profit, profit_percent,stk_size))
            self.orders[stk] = self.sell(data=stk,size=stk_size)
            self.hold[stk] = None
            #self.signal[stk]=0

        if profit_percent < self.p.stop_loss and stk_size>0:
            self.log('- 调出(止损): [%s], 盈亏:%.2f元(%.2f%%), 数量：(%d股)' % (stk._name, profit, profit_percent,stk_size))
            self.orders[stk] = self.sell(data=stk,size=stk_size)
            self.hold[stk] = None
        if profit_percent > self.p.stop_win and stk_size>0:
            self.log('- 调出(止盈): [%s], 盈亏:%.2f元(%.2f%%), 数量：(%d股)' % (stk._name, profit, profit_percent,stk_size))
            self.orders[stk] = self.sell(data=stk,size=stk_size)
            self.hold[stk] = None
            
        #死亡谷
        d_days=10
        arr_d_ma5=list(self.ma5[stk].get(ago=0,size=d_days))
        arr_d_ma10=list(self.ma10[stk].get(ago=0,size=d_days))
        arr_d_ma20=list(self.ma20[stk].get(ago=0,size=d_days))
        con_1d=False #条件1:过去10天内5日线下穿10日线
        con_2d=False #条件2:过去10天内5日线下穿20日线
        con_3d=False #条件3:过去10天内10日线下穿20日线
        con_4d=False #条件4:以上三个事件依次发生
        index_1d=0
        index_2d=0
        index_3d=0
        for i in range(d_days-1):
            if arr_d_ma5[i+1]-arr_d_ma10[i+1]<0 and arr_d_ma5[i]-arr_d_ma10[i]>0 and arr_d_ma5[i]>arr_d_ma5[i+1]:
                con_1d=True
                index_1d=i
            if arr_d_ma5[i+1]-arr_d_ma20[i+1]<0 and arr_d_ma5[i]-arr_d_ma20[i]>0 and arr_d_ma5[i]>arr_d_ma5[i+1]:
                con_2d=True
                index_2d=i
            if arr_d_ma10[i+1]-arr_d_ma20[i+1]<0 and arr_d_ma10[i]-arr_d_ma20[i]>0 and arr_d_ma10[i]>arr_d_ma10[i+1]:
                con_3d=True
                index_3d=i
        if(index_1d>index_2d>index_3d):
            con_4d=True
        if con_1d and con_2d and con_3d and con_4d:
            self.log('- 遭遇死亡谷，清仓: [%s], 盈亏:%.2f元(%.2f%%), 数量：(%d股)' % (stk._name, profit, profit_percent,stk_size))
            self.orders[stk] = self.sell(data=stk,size=stk_size)
            self.hold[stk] = None
            
        

                

In [83]:
# 比较两个大小相等的一维数组A和B，能否保证A中的元素全部大于B中的对应位置元素
def compArr(A,B):
    A=np.array(A)
    B=np.array(B)
    m=(A>B)
    return np.sum(m)==len(A)

class MakeData(bt.feeds.PandasData):
    lines = ('ma5','ma10','ma20','ma60','ma120','mayear',) 
    params = dict(
        ma5=-1,
        ma10=-1,
        ma20=-1,
        ma60=-1,
        ma120=-1,
        mayear=-1,
        dtformat = '%Y-%m-%d',
        openinterest = -1,
    )


class TestStrategy(bt.Strategy):
    params = dict(
        trailamount=0.0,
        trailpercent=0.05,
        # 止盈止损在这里设置
        stop_loss = -20,
        stop_win = 21,
        limit_percent = 1,
        # 注意设置最大持股数，这和买入的股数相关
        max_hold = int(stock_number*0.5),
        )
    
    def __init__(self):
       
        self.orders = dict()
        self.num = []
#         self.hold = dict()
        self.stk = self.datas[1:]
        self.hold = {}
        self.hold_stks = []
        self.d1 = dict()
        self.d2 = dict()
        self.macd = dict()
        self.hest = dict()
        self.ma5=dict()
        self.ma10=dict()
        self.ma20=dict()
        self.ma120=dict()
        self.mayear=dict()
        self.hest=dict()
        self.lost=dict()
        self.close=dict()
        for s in self.stk:
            self.orders[s] = None
#             self.signal[s] = 0
            self.hold[s] = None
            self.d1[s] = abs(s.ma20-s.ma120)
            self.d2[s] = abs(s.mayear-s.ma60)
            self.macd[s] = bt.indicators.MACDHisto(s.close,period_me1=12, period_me2=26, period_signal=9)
            self.ma5[s]=s.ma5
            self.ma10[s]=s.ma10
            self.ma20[s]=s.ma20
            self.ma120[s]=s.ma120
            self.mayear[s]=s.mayear
            self.close[s]=s.close
            self.hest[s] = bt.indicators.Highest(s.close,period=25)
            self.lost[s]=bt.indicators.Lowest(s.close,period=25)
            
            
    def log(self, arg):
        print('{} {}'.format(self.data0.datetime.date(), arg)) 
    
    def prenext(self):
#         self.log('prenext()')
        self.next()

    def nextstart(self):
#         self.log('nextstart()')
        self.next()
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:            
            return

        # 检查订单是否已经执行完成
        # 注意:如果账户资金不够，订单会被拒绝
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED,%.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            pass
#             self.log('Order Canceled/Margin/Rejected')
        for o in self.stk:
            self.orders[o] = None
    
    
    def next(self):
#         self.log('----next----')
        cur_date = self.data0.datetime.datetime()
        self.hold_stks = [b._name for b, pos in self.getpositions().items() if pos]
        self.num.append(len(self.hold_stks))
        for i,stk in enumerate(self.stk):  
# #             如果个股的日期和指数的时间不一致，跳出此次循环                
            if cur_date != stk.datetime.datetime():
                continue
            
            if self.orders[stk]:
                continue  
                
            pos = self.getposition(stk)
            val = self.broker.getvalue()
            cash = self.broker.get_cash()
            if not len(pos) and (len(self.hold_stks)<=self.p.max_hold):
                # 条件1:半年线和年线支撑股价——过去10天内股价均未跌破半年线、年线
                con1_days=10  # 考察年线和半年线的时间为10天
                con1_sum=0 # 计数变量
                arr_halfyear=list(self.ma120[stk].get(ago=0,size=con1_days))
                arr_year=list(self.mayear[stk].get(ago=0,size=con1_days))
                arr_close=list(self.close[stk].get(ago=0,size=con1_days))
                if(len(arr_halfyear)<con1_days):
                    con1=False
                else:
                    for i in range(con1_days):
                        if max(arr_halfyear[i],arr_year[i])<arr_close[i]:
                            con1_sum+=1
                    con1=con1_sum==con1_days
                # 条件2:5日线和20日线粘连、多次穿插——理解为：往前数两天，过去20天里5日线和20日线差不超过当前股价10%，且穿插>=1次
                con2_days=20
                con2_sum=0  # 穿插次数
                con2_sum2=0  # 两条均线差值
                arr_5=list(self.ma5[stk].get(ago=2,size=con2_days))
                arr_20=list(self.ma20[stk].get(ago=2,size=con2_days))
                arr_close2=list(self.close[stk].get(ago=2,size=con2_days))
                if(len(arr_5)<con2_days):
                    con2=False
                else:
                    for i in range(1,con2_days):
                        if (arr_5[i]-arr_20[i])*(arr_5[i-1]-arr_20[i-1])<0: # 连续两天5日线和20日线的差值不同符号，代表发生了一次穿插
                            con2_sum+=1
                        if abs(arr_5[i]-arr_20[i])<arr_close2[i]*0.15 : # 5日线和20日线相差不大
                            con2_sum2+=1
                    con2= con2_sum>=1 and con2_sum2==con2_days-1
                # 条件3:股价在一较小幅度内震荡——理解为：过去10天里最大股价和最小股价差不超过当前股价的20%
                con3_days=10
                arr_close3=list(self.close[stk].get(ago=2,size=con3_days))
                if(len(arr_close3)<con3_days):
                    con3=False
                else:
                    con3=(max(arr_close3)-min(arr_close3))/max(arr_close3)<0.2
                # 条件4:5日线上穿最后一条均线，形成明确的上升趋势
                con4=stk.ma5[0]>max(stk.ma20[0],stk.ma60[0],stk.ma120[0],stk.mayear[0])
                #con4=stk.ma5[-1]<max(stk.ma20[-1],stk.ma60[-1],stk.ma120[-1],stk.mayear[-1]) and stk.ma5[0]>max(stk.ma20[0],stk.ma60[0],stk.ma120[0],stk.mayear[0]) and stk.ma5[0]>stk.ma5[-1]
                
                # 条件5:多头排列，均线呈现上升趋势
                con5=(stk.close[0]>stk.ma5[0] and stk.ma5[0]>stk.ma20[0] and stk.ma20[0]>stk.ma60[0]) and (stk.close[-1]<stk.close[0] and stk.ma5[-1]<stk.ma5[0] and stk.ma20[-1]<stk.ma20[0] and stk.ma60[-1]<stk.ma60[0])
                
                # 条件6:两次调整，第二次调整未跌破第一次低点的110%，且第二次调整后的反弹超过第一次高点
                arr_highest=list(self.hest[stk].get(ago=0,size=2))
                arr_lowest=list(self.lost[stk].get(ago=0,size=2))
                if(len(arr_highest)==2 and len(arr_lowest)==2):
                    # 注意：为保障不碰到现在时间离最后一个高点比离最后一个低点更近的情况，只需保证股价高于前一天。
                    con6=stk.close[0]>stk.close[-1] and stk.close[0]>arr_highest[0] and arr_lowest[1]>arr_lowest[0]*1.1
                else:
                    con6=False
                
                
                if (con1 and con2 and con3 and con4) or (con5 and con6):
                    target_money = val / self.p.max_hold
                    target_size = math.floor(target_money/stk.close[0]/100)
        
                    limit_price = stk.close[0] * (1.0 - self.p.limit_percent / 100.0)
                
                    self.log('BUY CREATE: [%s], %.2f,%d只' % (stk._name,stk.close[0],100*target_size))
                    self.orders[stk] = self.buy(data=stk,size=100*target_size)
                    self.hold[stk] = stk.datetime.datetime()
                    
                    
            else:
                  sell_stk(self,stk) 
     
                    
    def stop(self):
        self.log('最终总资产:%.2f元, 盈利:%.2f元 ,最大持有标的数量:%d个'% (self.broker.getvalue(), self.broker.getvalue() - g_init_cash, max(self.num)))
                    
        
              
        
if __name__ == '__main__':
    # 创建一个大脑
    cerebro = bt.Cerebro()
    start_date,end_date = '20200101','20201231'
    prepare_index_data(start_date,end_date)
    print('*'*100)
    stock_data = prepare_stk_data(codes,start_date,end_date)
    print('*'*100)
#     stock_data = del_small(stock_data,start_date,end_date)
#     ticker = 0
#     print(stock_data.keys())
    print('开始添加数据……')
    for nm,df in stock_data.items():
        datad = MakeData(
            dataname=df,
            datetime = 'trade_date',
            open='open',
            high='high',
            low='low',
            close='close',
            volume='volume',
            ma5 = 'ma_5',
            ma10='ma_10',
            ma20 ='ma_20',
            ma60 = 'ma_60',
            ma120 ='ma_120',
            mayear = 'ma_year',
            openinterest=-1,
            timeframe=bt.TimeFrame.Days,
           )
        cerebro.adddata(datad, name=nm)
    print('添加完毕！')
    
                                                     
    
   
    # 设置初始资金
    cerebro.broker.setcash(g_init_cash)
    # 设置佣金
    cerebro.broker.setcommission(commission=0.0015)
    # # 设置是否cheat on close (next bar)
    cerebro.broker.set_coc(True)
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='AnnualReturn')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown')
    cerebro.addanalyzer(bt.analyzers.TimeDrawDown, _name='TimeDrawDown')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')
#     cerebro.addanalyzer(TradeAnalyzer, _name='trade_list')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='Trade')
    cerebro.addstrategy(TestStrategy)
 
    # 打印初始资金
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())   
    # 运行所有程序
    strats = cerebro.run(tradehistory=True)
#     trade_list = strats[0].analyzers.trade_list.get_analysis()
    print('Sharpe Ratio:', strats[0].analyzers.SharpeRatio.get_analysis(),
         '\n AnnualReturn:', strats[0].analyzers.AnnualReturn.get_analysis(),
    # #       # '\nCalmar:', strats[0].analyzers.Calmar.get_analysis(),
    # #       # '\nVWR:', strats[0].analyzers.VWR.get_analysis(),
    # #       '\nTransactions:', strats[0].analyzers.Transactions.get_analysis(),
    # #       #'\n' + tabulate(strats[0].analyzers.TradeAnalyzer.get_analysis(), headers="keys") + '\n'
         # '\nTimeReturn:', strats[0].analyzers.TimeReturn.get_analysis(),
    # #       # '\n_SQN:', strats[0].analyzers._SQN.get_analysis(),
    # #       # '\nPeriodStats:', strats[0].analyzers.PeriodStats.get_analysis(),
    # #       # '\nLogReturnsRolling:', strats[0].analyzers.LogReturnsRolling.get_analysis(),
    # #       # '\nPositionsValue:', strats[0].analyzers.PositionsValue.get_analysis(),
         # '\npnl:', strats[0].analyzers.pnl.get_analysis(),
         '\nTimeDrawDown:', strats[0].analyzers.TimeDrawDown.get_analysis(),
         '\nDrawDown:', strats[0].analyzers.DrawDown.get_analysis(),
          '\nPnl:', strats[0].analyzers.Trade.get_analysis()['won'],
           '\nPnl:', strats[0].analyzers.Trade.get_analysis()['lost'],
       )
#     print('\n' + tabulate(trade_list, headers="keys") + '\n')
    
    # 打印最终结果
#     print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


****************************************************************************************************
准备数据中……
****************************************************************************************************
开始添加数据……
添加完毕！
Starting Portfolio Value: 1000000.00
2020-02-04 BUY CREATE: [002271.SZ], 26.59,700只
2020-02-04 BUY CREATE: [002353.SZ], 38.21,500只
2020-02-04 BUY CREATE: [300558.SZ], 68.22,200只
2020-02-04 BUY CREATE: [603727.SH], 19.55,1000只
2020-02-04 BUY CREATE: [600584.SH], 24.30,800只
2020-02-04 BUY CREATE: [300463.SZ], 31.22,600只
2020-02-04 BUY CREATE: [002212.SZ], 20.41,900只
2020-02-05 BUY EXECUTED,26.59
2020-02-05 BUY EXECUTED,38.21
2020-02-05 BUY EXECUTED,68.22
2020-02-05 BUY EXECUTED,19.55
2020-02-05 BUY EXECUTED,24.30
2020-02-05 BUY EXECUTED,31.22
2020-02-05 BUY EXECUTED,20.41
2020-02-12 - 调出(止盈): [002212.SZ], 盈亏:4239.00元(23.08%), 数量：(900股)
2020-02-13 SELL EXECUTED, 25.12
2020-02-14 - 调出(止盈): [600584.SH], 盈亏:4840.00元(24.90%), 数量：(800股)
2020-02-17 SELL EXECUTED, 30.35
2020

2020-07-15 SELL EXECUTED, 19.99
2020-07-15 BUY EXECUTED,5.39
2020-07-15 SELL EXECUTED, 72.06
2020-07-15 BUY EXECUTED,13.72
2020-07-15 SELL EXECUTED, 6.41
2020-07-15 - 调出(止盈): [600605.SH], 盈亏:4968.00元(24.89%), 数量：(1800股)
2020-07-15 BUY CREATE: [600598.SH], 18.52,1100只
2020-07-15 BUY CREATE: [002353.SZ], 31.36,600只
2020-07-15 BUY CREATE: [002423.SZ], 11.39,1900只
2020-07-16 SELL EXECUTED, 13.85
2020-07-16 BUY EXECUTED,18.52
2020-07-16 BUY EXECUTED,31.36
2020-07-16 BUY EXECUTED,11.39
2020-07-16 BUY CREATE: [600605.SH], 12.48,1600只
2020-07-17 BUY EXECUTED,12.48
2020-07-17 BUY CREATE: [600500.SH], 5.60,3700只
2020-07-17 BUY CREATE: [000625.SZ], 10.73,1900只
2020-07-17 BUY CREATE: [000798.SZ], 6.09,3400只
2020-07-17 BUY CREATE: [600262.SH], 20.52,1000只
2020-07-17 BUY CREATE: [002249.SZ], 3.85,5500只
2020-07-17 BUY CREATE: [600683.SH], 4.20,5000只
2020-07-17 BUY CREATE: [002807.SZ], 4.48,4700只
2020-07-17 BUY CREATE: [000823.SZ], 13.30,1500只
2020-07-20 BUY EXECUTED,5.60
2020-07-20 BUY EXECUTED,10.73

2020-10-23 SELL EXECUTED, 34.10
2020-10-23 BUY CREATE: [000791.SZ], 3.53,6300只
2020-10-23 BUY CREATE: [002737.SZ], 15.99,1300只
2020-10-23 - 调出(止盈): [300479.SZ], 盈亏:6721.00元(30.83%), 数量：(1300股)
2020-10-23 BUY CREATE: [300471.SZ], 14.32,1500只
2020-10-23 BUY CREATE: [300577.SZ], 33.23,600只
2020-10-23 BUY CREATE: [601677.SH], 14.26,1500只
2020-10-23 BUY CREATE: [300343.SZ], 3.98,5600只
2020-10-23 BUY CREATE: [600158.SH], 12.63,1700只
2020-10-26 BUY EXECUTED,3.53
2020-10-26 BUY EXECUTED,15.99
2020-10-26 SELL EXECUTED, 21.94
2020-10-26 BUY EXECUTED,14.32
2020-10-26 BUY EXECUTED,33.23
2020-10-26 BUY EXECUTED,14.26
2020-10-26 BUY EXECUTED,3.98
2020-10-26 - 调出(止损): [002818.SZ], 盈亏:-4537.00元(-20.92%), 数量：(1300股)
2020-10-27 SELL EXECUTED, 13.19
2020-10-29 - 调出(止盈): [000729.SZ], 盈亏:4650.00元(21.03%), 数量：(3000股)
2020-10-29 - 调出(止损): [600095.SH], 盈亏:-4498.00元(-20.87%), 数量：(1300股)
2020-10-30 SELL EXECUTED, 8.92
2020-10-30 SELL EXECUTED, 13.12
2020-10-30 - 调出(止损): [002353.SZ], 盈亏:-4152.00元(-20.56%), 数量：(6